## Submission Notebook Template 

<b> Install your packages below: </b>

In [ ]:
!pip install git+https://github.com/Total-RD/pymgrid/
## Other packages 
!pip install prettytable

In the section below, you must run your methodology for solving the problem from start to finish :

In [ ]:
import pickle

"""
The buildings mentionned below are specific to the hackathon and are not available in this repo.
You can replace them with any MicroGrid object generated from pymgrid
"""

with open('building_1.pkl', 'rb') as f:
    building_1 = pickle.load(f)
    building_1.train_test_split()

with open('building_2.pkl', 'rb') as f:
    building_2 = pickle.load(f)
    building_2.train_test_split()
    
with open('building_3.pkl', 'rb') as f:
    building_3 = pickle.load(f)
    building_3.train_test_split()

buildings = [building_1, building_2, building_3]

<h2> Evaluation for Rule Based Approaches </h2>

<b> 1) Import all used libraries and scripts here </b>

In [ ]:
import time # Necessary to evaluate frugality
import json # Necessary to export your results
## Other packages 
import prettytable

<b> 2) Implementation of the rules that generate control dictionaries </b>

In [ ]:
def naive_rule_based_strategy(building):
    
    building_data = building.get_updated_values()

    total_building_cost = 0

    while building.done == False:
        
        load = building_data['load']
        pv = building_data['pv']
        capa_to_charge = building_data['capa_to_charge']
        capa_to_dischare = building_data['capa_to_discharge']
        grid_status = building_data['grid_status']
        
        if load - pv >=  0:
            if grid_status == 1:
                control_dict = {
                    'battery_charge': 0,
                    'battery_discharge': min(capa_to_dischare, load-pv),
                    'grid_import': max(0, load-pv-min(capa_to_dischare, load-pv)),
                    'grid_export':0,
                    'pv_consummed':pv,
                    'genset':0,
                }
            else:
                control_dict = {
                    'battery_charge': 0,
                    'battery_discharge': min(capa_to_dischare, load-pv),
                    'grid_import': 0,
                    'grid_export':0,
                    'pv_consummed':pv,
                    'genset':max(0, load-pv-min(capa_to_dischare, load-pv)),
               }
        else:
            control_dict = {
                'battery_charge': min(pv-load,capa_to_charge),
                'battery_discharge': 0,
                'grid_import': 0,
                'grid_export':pv-load-min(pv-load,capa_to_charge),
                'pv_consummed':pv,
                'genset':0,
            } 
            
        building_data = building.run(control_dict)
        total_building_cost += building.get_cost()
    
    return total_building_cost

<b> 3) Run of the rules on the Test environment </b>

In [ ]:
"""
Rule based methods have no "training" as such, this means Training CPU Time will always be 0 and only
Test CPU Time will represent frugality
"""

eval_start = time.process_time()

total_building_costs = []

for building in buildings:

    building.reset(testing = True)

    total_building_cost = naive_rule_based_strategy(building)
    total_building_costs.append(total_building_cost)

eval_end = time.process_time()



In [ ]:
total_cost_building_1 = total_building_costs[0]
total_cost_building_2 = total_building_costs[1]
total_cost_building_3 = total_building_costs[2]

In [ ]:
frugality = eval_end - eval_start

<b> 4) Store & Export Results in JSON format </b>

In [ ]:
final_results = {
    "building_1_performance" : total_cost_building_1,
    "building_2_performance" : total_cost_building_2,
    "building_3_performance" : total_cost_building_3,
    "frugality" : frugality,
}

# print(list(final_results.items()))

x = prettytable.PrettyTable()
x.title = "Results of the rule-based approach"
x.field_names = ["Metric", "Value"]
for el in list(final_results.items()):
    x.add_row(el)
print(x)